<a href="https://colab.research.google.com/github/GiX007/agent-labs/blob/main/06_autoGen/01_Multi_Agent_Conversation_and_Stand_up_Comedy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Agent Conversation and Stand-up Comedy

## Setup

In [ ]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

## Define an AutoGen agent

In [ ]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [ ]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

Sure, here it is: Why couldn't the bicycle stand up by itself? Because it was two tired!


In [ ]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

Of course! Just let me know which joke you'd like me to repeat.


As we see, there is no memory.

## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [ ]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [ ]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Thanks, Joe! I'm always ready to bring the laughs. Did you hear about the claustrophobic astronaut? He just needed a little space!

--------------------------------------------------------------------------------
joe (to cathy):

That astronaut must have felt right at home in a crowded elevator.

--------------------------------------------------------------------------------
cathy (to joe):

Exactly, Joe! He'd probably be like, "Ah, finally, some breathing room!"

--------------------------------------------------------------------------------


## Print some results

We can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [ ]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'role': 'assistant'},
 {'content': "Thanks, Joe! I'm always ready to bring the laughs. Did you hear "
             'about the claustrophobic astronaut? He just needed a little '
             'space!',
  'role': 'user'},
 {'content': 'That astronaut must have felt right at home in a crowded '
             'elevator.',
  'role': 'assistant'},
 {'content': 'Exactly, Joe! He\'d probably be like, "Ah, finally, some '
             'breathing room!"',
  'role': 'user'}]


In [ ]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 63,
                                                             'cost': 0.0001985,
                                                             'prompt_tokens': 208,
                                                             'total_tokens': 271},
                                      'total_cost': 0.0001985},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 63,
                                                             'cost': 0.0001985,
                                                             'prompt_tokens': 208,
                                                             'total_tokens': 271},
                                      'total_cost': 0.0001985}}


In [ ]:
pprint.pprint(chat_result.summary)

'Exactly, Joe! He\'d probably be like, "Ah, finally, some breathing room!"'


## Get a better summary of the conversation

In [ ]:
chat_result = joe.initiate_chat(
    cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Thanks, Joe! I'm always ready to bring the laughs. Did you hear about the claustrophobic astronaut? He just needed a little space!

--------------------------------------------------------------------------------
joe (to cathy):

That astronaut must have felt right at home in a crowded elevator.

--------------------------------------------------------------------------------
cathy (to joe):

Exactly, Joe! He'd probably be like, "Ah, finally, some breathing room!"

--------------------------------------------------------------------------------


In [ ]:
pprint.pprint(chat_result.summary)

('The conversation involved sharing a joke about a claustrophobic astronaut '
 'and making a related witty comment.')


## Chat Termination

Chat can be terminated using a termination conditions.

In [ ]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [ ]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe, glad you're enjoying the jokes! Alright, here's one for you: Why did the math book look sad? Because it had too many problems!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, I love a good math joke, Cathy! Maybe the math book just needed some... addition-al help!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, nice one, Joe! Looks like you know how to sum up a joke!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, thanks, Cathy! I try to divide my time between writing jokes and making people laugh!

--------------------------------------------------------------------------------
cathy (to joe):

Well, Joe, it sounds like you're really adding up those

In [ ]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

We were on a roll with math jokes, talking about adding up laughs and multiplying good vibes!

--------------------------------------------------------------------------------
